In [2]:
import pandas as pd
import numpy as np

In [13]:
df_cleaned = pd.read_csv("olist_cleaned.csv", parse_dates=["order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date", "order_delivered_customer_date", "order_estimated_delivery_date"])

In [14]:
olist = df_cleaned[["customer_unique_id","order_id","order_status","order_purchase_timestamp"]]

In [15]:
#sorting my  values by order_purchase_timestamp
olist = olist.sort_values('order_purchase_timestamp')
#counting purchase by customer
purchase_count = olist.groupby("customer_unique_id")["order_id"].count()
#my repeat customers 
repeat_customer = purchase_count.apply(lambda x : 1 if x > 1 else 0 )
olist = olist.merge(repeat_customer.rename("target"), on = "customer_unique_id")

In [16]:
olist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118307 entries, 0 to 118306
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   customer_unique_id        118307 non-null  object        
 1   order_id                  118307 non-null  object        
 2   order_status              118307 non-null  object        
 3   order_purchase_timestamp  118307 non-null  datetime64[ns]
 4   target                    118307 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 4.5+ MB


In [17]:
olist["purchase_hour"] = olist["order_purchase_timestamp"].dt.hour
olist["purchase_day"] = olist["order_purchase_timestamp"].dt.day_of_week
olist["purchase_month"] = olist["order_purchase_timestamp"].dt.month

olist["order_status_encoded"] = olist["order_status"].astype("category").cat.codes

In [18]:
features = (
    olist.groupby("customer_unique_id")
         .agg({
             "purchase_hour": "mean",
             "purchase_day": "mean",
             "purchase_month": "mean",
            "order_status_encoded": "mean"})
         .reset_index())


In [19]:
feature = features.merge(olist[["customer_unique_id","target"]], on= "customer_unique_id")

In [20]:
feature.head()

,customer_unique_id,purchase_hour,purchase_day,purchase_month,order_status_encoded,target
0,0000366f3b9a7992bf8c76cfdf3221e2,10.0,3.0,5.0,2.0,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,11.0,0.0,5.0,2.0,0
2,0000f46a3911fa3c0805444483337064,21.0,4.0,3.0,2.0,0
3,0000f6ccb0745a6a4b88665a16c9f078,20.0,3.0,10.0,2.0,0
4,0004aac84e0df4da2b147fca70cf8255,19.0,1.0,11.0,2.0,0


In [21]:
x = feature.drop(["customer_unique_id", "target"], axis= 1)
y = feature["target"]

In [23]:
!pip install joblib


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
pip install scikitlearn

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement scikitlearn (from versions: none)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for scikitlearn


In [ ]:
import joblib
import sklearn.metrics as sm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y , test_size= 0.2 , random_state=42)

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)


In [ ]:
log_reg = LogisticRegression(class_weight="balanced", random_state=42 )
log_reg.fit(x_train_scaled , y_train)

In [ ]:
y_predict = log_reg.predict(x_test_scaled)
conf_metric = sm.confusion_matrix(y_test, y_predict)
accuracy_score = sm.accuracy_score(y_test , y_predict)
precision_score = sm.precision_score(y_test , y_predict)
recall_score =  sm.recall_score(y_test, y_predict)
f1_score = sm.f1_score(y_test , y_predict)
classf_report = sm.classification_report(y_test , y_predict)

In [ ]:
print(f"Accuracy: {accuracy_score:.2f}")
print(f"Precision: {precision_score:.2f}")
print(f"Recall: {recall_score:.2f}")
print(f"F1-Score: {f1_score:.2f}")
print("Classification Report:")
print(sm.classification_report(y_test, y_predict))

In [ ]:
print("Confusion Matrix:")
print(conf_metric)
plt.figure(figsize = (10,7))
sns.heatmap(conf_metric, annot = True)
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

using Random Forest

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
rf = RandomForestClassifier(n_estimators=100 , max_depth=5 , random_state=42, n_jobs=1, class_weight="balanced")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
rf.fit(x_train, y_train)

# Predict
y_pred_rf = rf.predict(x_test)

conf_metric_rf = sm.confusion_matrix(y_test, y_pred_rf)
accuracy_score_rf = sm.accuracy_score(y_test , y_pred_rf)
precision_score_rf = sm.precision_score(y_test , y_pred_rf)
recall_score_rf =  sm.recall_score(y_test, y_pred_rf)
f1_score_rf = sm.f1_score(y_test , y_pred_rf)
classf_report_rf = sm.classification_report(y_test , y_pred_rf)

In [ ]:
print(f"Accuracy: {accuracy_score_rf:.2f}")
print(f"Precision: {precision_score_rf:.2f}")
print(f"Recall: {recall_score_rf:.2f}")
print(f"F1-Score: {f1_score_rf:.2f}")
print("Classification Report:")
print(sm.classification_report(y_test, y_pred_rf))

In [ ]:
print("Confusion Matrix:")
print(conf_metric_rf)
plt.figure(figsize = (10,7))
sns.heatmap(conf_metric, annot = True)
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

Using XG_BOOST

In [ ]:
#Calculate scale_pos_weight
neg, pos = np.bincount(y_train)   # counts of 0s and 1s in training
scale_pos_weight = neg / pos

# Train XGBoost with imbalance handling
xg = XGBClassifier(
    scale_pos_weight=scale_pos_weight,
    learning_rate=0.05,
    n_estimators=500,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric="logloss",

)

xg.fit(x_train, y_train)

# Predictions
y_proba = xg.predict_proba(x_test)[:,1]
y_pred_xg = (y_proba >= 0.44).astype(int)


In [ ]:
conf_metric_xg = sm.confusion_matrix(y_test, y_pred_xg)
accuracy_score_xg = sm.accuracy_score(y_test , y_pred_xg)
precision_score_xg= sm.precision_score(y_test , y_pred_xg)
recall_score_xg =  sm.recall_score(y_test, y_pred_xg)
f1_score_xg = sm.f1_score(y_test , y_pred_xg)
classf_report_xg = sm.classification_report(y_test , y_pred_xg)

In [ ]:
print(f"Accuracy: {accuracy_score_xg:.2f}")
print(f"Precision: {precision_score_xg:.2f}")
print(f"Recall: {recall_score_xg:.2f}")
print(f"F1-Score: {f1_score_xg:.2f}")
print("Classification Report:")
print(sm.classification_report(y_test, y_pred_xg))

In [ ]:
print("Confusion Matrix:")
print(conf_metric_xg)
plt.figure(figsize = (10,7))
sns.heatmap(conf_metric, annot = True)
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

# Plot
plt.figure(figsize=(7,6))
plt.plot(fpr, tpr, color="darkorange", lw=2, 
         label=f"XGBoost ROC curve (AUC = {roc_auc:.3f})")
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.grid(True, linestyle="--", alpha=0.7)
plt.show()


Predictive Modelling using XG_BOOST

In [ ]:
# import pickle

# # Save XGBoost model
# with open('XGBoost.pkl', 'wb') as f:
#     pickle.dump(xg, f)

# #with open('random_forest_model.pkl', 'rb') as f:
#    #  loaded_rf = pickle.load(f)